In [1]:
from datetime import datetime, timedelta
from collections import defaultdict

import pandas as pd
import numpy as np


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df_users = pd.read_csv('../data/random-users.csv')

In [4]:
df_logs = pd.read_csv('../data/random-logs.csv')

In [5]:
df_logs.head()

,timestamp,user_id,action_category,action_type,duration_seconds
0,2024-01-01 22:02:12.273771,USR1835,user_management,assign_roles,83
1,2024-01-02 00:24:02.737941,USR1423,assessment,grade_submission,74
2,2024-01-02 00:28:19.356219,USR1016,platform_exploration,view_features,249
3,2024-01-02 02:34:12.448593,USR1730,platform_exploration,view_features,73
4,2024-01-02 03:12:45.202030,USR1125,platform_exploration,view_features,222


In [6]:

def preprocess_required_features(df):
    """
    Preprocess the required_features column by splitting comma-separated strings
    into multiple binary features that can be handled by DictVectorizer.
    
    Parameters:
    df: DataFrame containing the required_features column
    
    Returns:
    DataFrame with required_features transformed into a dictionary format
    compatible with DictVectorizer
    """
    df = df.copy()
    
    def process_feature_string(feature_string):
        """
        Convert comma-separated string into a dictionary of binary features.
        Each feature will be prefixed with 'required_feature_' to avoid potential
        naming conflicts with other features.
        """
        features = [f.strip() for f in feature_string.split(',')]
        return {f'required_feature_{feature}': 1 for feature in features}
    
    # Convert required_features strings to dictionaries
    feature_dicts = df['required_features'].apply(process_feature_string)
    
    # Remove original required_features column
    df = df.drop('required_features', axis=1)
    
    # Convert other columns to dictionary format for DictVectorizer
    record_dicts = df.to_dict('records')
    
    # Merge the feature dictionaries with the rest of the records
    for record, feature_dict in zip(record_dicts, feature_dicts):
        record.update(feature_dict)
    
    return record_dicts

In [7]:
def create_log_features(df_users, df_logs, cutoff_date):
    """
    Create aggregated features from logs data up to the cutoff_date to avoid data leakage.
    
    Parameters:
    df_users: DataFrame with user information
    df_logs: DataFrame with user activity logs
    cutoff_date: datetime to filter log data
    
    Returns:
    DataFrame with user_id and engineered features
    """
    # Filter logs up to cutoff_date
    df_logs_filtered = df_logs[df_logs['timestamp'] < cutoff_date].copy()
    
    # Basic engagement metrics
    engagement_metrics = df_logs_filtered.groupby('user_id').agg({
        'timestamp': ['count', 'nunique'],  # Total actions and unique days
        'duration_seconds': ['sum', 'mean', 'std']  # Time spent metrics
    }).round(2)
    
    # Flatten column names
    engagement_metrics.columns = [
        'total_actions',
        'active_days',
        'total_duration',
        'avg_duration',
        'std_duration'
    ]
    
    # Action category distribution
    category_counts = df_logs_filtered.groupby(['user_id', 'action_category']).size().unstack(
        fill_value=0
    ).add_prefix('category_')
    
    # Action type distribution (top 10 most common)
    top_actions = df_logs_filtered['action_type'].value_counts().nlargest(10).index
    action_counts = df_logs_filtered[df_logs_filtered['action_type'].isin(top_actions)]\
        .groupby(['user_id', 'action_type']).size().unstack(fill_value=0).add_prefix('action_')
    
    # Time-based features
    df_logs_filtered['hour'] = df_logs_filtered['timestamp'].dt.hour
    time_metrics = df_logs_filtered.groupby('user_id').agg({
        'hour': lambda x: len(x[x.between(9, 17)]) / len(x)  # Fraction of activity during business hours
    }).round(2)
    time_metrics.columns = ['business_hours_ratio']
    
    # Activity patterns
    df_logs_filtered['days_since_signup'] = (
        df_logs_filtered['timestamp'] - 
        df_logs_filtered['user_id'].map(df_users.set_index('user_id')['signup_date'])
    ).dt.days
    
    recency_metrics = df_logs_filtered.groupby('user_id').agg({
        'days_since_signup': ['min', 'max']
    }).round(2)
    recency_metrics.columns = ['days_to_first_action', 'days_to_last_action']
    
    # Advanced engagement metrics
    df_logs_filtered['prev_timestamp'] = df_logs_filtered.groupby('user_id')['timestamp'].shift(1)
    df_logs_filtered['time_between_actions'] = (
        df_logs_filtered['timestamp'] - df_logs_filtered['prev_timestamp']
    ).dt.total_seconds() / 3600  # Convert to hours
    
    engagement_patterns = df_logs_filtered.groupby('user_id').agg({
        'time_between_actions': ['mean', 'std']
    }).round(2)
    engagement_patterns.columns = ['avg_hours_between_actions', 'std_hours_between_actions']
    
    # Feature importance indicators
    feature_exploration = df_logs_filtered[
        df_logs_filtered['action_type'] == 'view_features'
    ].groupby('user_id').size().to_frame('feature_views')
    
    # Combine all features
    log_features = pd.concat([
        engagement_metrics,
        category_counts,
        action_counts,
        time_metrics,
        recency_metrics,
        engagement_patterns,
        feature_exploration
    ], axis=1).reset_index()
    
    # Fill NaN values with 0 for new users or users with missing metrics
    log_features = log_features.fillna(0)
    
    return log_features


def prepare_lead_scoring_data(df_users, df_logs, train_end_date, val_end_date):
    """
    Prepare train, validation, and test datasets with log-based features.
    
    Parameters:
    df_users: DataFrame with user information
    df_logs: DataFrame with log data
    train_end_date: str, cutoff date for training data
    val_end_date: str, cutoff date for validation data
    
    Returns:
    tuple of (train, validation, test) DataFrames
    """
    # Make a copy of the dataframe to avoid modifying the original
    df_users = df_users.copy()
    df_logs = df_logs.copy()
    
    # Convert signup_date to datetime if it's not already
    df_users['signup_date'] = pd.to_datetime(df_users['signup_date'])
    df_logs['timestamp'] = pd.to_datetime(df_logs['timestamp'])
    
    # Convert dates to datetime
    train_end_date = pd.to_datetime(train_end_date)
    val_end_date = pd.to_datetime(val_end_date)
    
    # Create masks for time-based splitting
    train_mask = df_users['signup_date'] < train_end_date
    val_mask = (df_users['signup_date'] >= train_end_date) & (df_users['signup_date'] < val_end_date)
    test_mask = df_users['signup_date'] >= val_end_date
    
    # Split user data
    df_train = df_users[train_mask].copy()
    df_val = df_users[val_mask].copy()
    df_test = df_users[test_mask].copy()
    
    # Create features for each split using appropriate cutoff dates
    train_features = create_log_features(df_users, df_logs, train_end_date)
    val_features = create_log_features(df_users, df_logs, val_end_date)
    test_features = create_log_features(df_users, df_logs, df_logs['timestamp'].max())

    # Merge features with user data
    df_train = df_train.merge(train_features, on='user_id', how='left')
    df_val = df_val.merge(val_features, on='user_id', how='left')
    df_test = df_test.merge(test_features, on='user_id', how='left')

    return df_train, df_val, df_test

In [8]:
df_train, df_val, df_test = prepare_lead_scoring_data(
    df_users,
    df_logs,
    train_end_date='2024-03-01',
    val_end_date='2024-03-15'
)

In [9]:
from sklearn.feature_extraction import DictVectorizer

def prepare_features(df, dv=None, fit=True):
    """
    Prepare features using DictVectorizer, excluding ID and date columns

    Parameters:
    df: pandas DataFrame
    dv: DictVectorizer instance (optional)
    fit: boolean, whether to fit the DictVectorizer or just transform
    
    Returns:
    X: sparse matrix of features
    dv: fitted DictVectorizer
    feature_names: list of feature names
    """
    # Create a copy to avoid modifying the original
    df = df.copy()
    
    # Identify columns to exclude
    date_columns = ['signup_date', 'conversion_date']
    exclude_columns = ['user_id', 'converted'] + date_columns
    
    # Convert DataFrame to list of dictionaries, excluding specified columns
    df = df.drop(columns=exclude_columns)
    df = df.fillna(0)
    feature_dict = preprocess_required_features(df)
    
    # Initialize DictVectorizer if not provided
    if dv is None:
        dv = DictVectorizer(sparse=True)
    
    # Fit or transform
    if fit:
        X = dv.fit_transform(feature_dict)
    else:
        X = dv.transform(feature_dict)
    
    return X, dv

X_train, dv = prepare_features(df_train, fit=True)
X_val, _ = prepare_features(df_val, dv=dv, fit=False)
X_test, _ = prepare_features(df_test, dv=dv, fit=False)

# Get target variable
y_train = df_train['converted'].values
y_val = df_val['converted'].values
y_test = df_test['converted'].values

In [10]:
# Convert sparse matrix to dense for the first 5 rows
X_dense = X_train[:5].toarray()

# Create DataFrame with feature names
encoded_df = pd.DataFrame(X_dense, columns=dv.feature_names_)

In [11]:
pd.set_option('display.max_rows', None)

In [12]:
encoded_df.T

,0,1,2,3,4
action_configure_dashboard,0.00,0.00,0.00,0.00,0.00
action_create_assignment,0.00,0.00,0.00,0.00,0.00
action_create_course,0.00,0.00,0.00,0.00,0.00
action_grade_submission,0.00,0.00,1.00,0.00,0.00
action_invite_users,0.00,0.00,0.00,0.00,0.00
action_organize_content,1.00,0.00,0.00,1.00,0.00
action_publish_course,0.00,0.00,0.00,1.00,0.00
action_record_session,0.00,0.00,0.00,0.00,0.00
action_setup_integration,0.00,2.00,0.00,0.00,0.00
action_view_features,14.00,20.00,25.00,17.00,26.00


In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Initialize and train the model
model = LogisticRegression(max_iter=1000, random_state=1)
model.fit(X_train, y_train)

# Get predictions
train_pred = model.predict_proba(X_train)[:, 1]
val_pred = model.predict_proba(X_val)[:, 1]

# Calculate AUC
train_auc = roc_auc_score(y_train, train_pred)
val_auc = roc_auc_score(y_val, val_pred)

print(f'Train AUC: {train_auc:.3f}')
print(f'Validation AUC: {val_auc:.3f}')

# Let's also look at the most important features
feature_importance = pd.DataFrame({
    'feature': dv.feature_names_,
    'importance': model.coef_[0],
    'abs_importance': np.abs(model.coef_[0])
})
feature_importance = feature_importance.sort_values('abs_importance', ascending=False)

print('\nTop 10 most important features:')
print(feature_importance.head(10))

Train AUC: 0.714
Validation AUC: 0.557

Top 10 most important features:
                               feature  importance  abs_importance
41    lead_source=Education Conference   -0.328239        0.328239
38         expected_student_count=<100    0.327050        0.327050
44                lead_source=Referral   -0.316123        0.316123
60  primary_use_case=Employee Training   -0.310061        0.310061
57   primary_use_case=Academic Courses    0.296707        0.296707
50            organization_size=51-200    0.281370        0.281370
8             action_setup_integration    0.270649        0.270649
4                  action_invite_users    0.268070        0.268070
37     expected_student_count=501-1000   -0.264156        0.264156
48           organization_size=201-500   -0.251970        0.251970


/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Define parameter grid
param_grid = {
    'C': [0.0001, 0.001, 0.01, 0.1, 1, 10],
    'solver': ['liblinear', 'saga'],
    'penalty': ['l1', 'l2']
}

# Try each combination and track results
results = []

for C in param_grid['C']:
    for solver in param_grid['solver']:
        for penalty in param_grid['penalty']:
            # Skip invalid combinations
            if solver == 'liblinear' and penalty not in ['l1', 'l2']:
                continue

            print(C, solver, penalty)
            # Train model
            model = LogisticRegression(
                C=C, 
                solver=solver,
                penalty=penalty,
                random_state=1,
                max_iter=1000
            )
            model.fit(X_train, y_train)
            
            # Get predictions
            train_pred = model.predict_proba(X_train)[:, 1]
            val_pred = model.predict_proba(X_val)[:, 1]
            
            # Calculate AUC
            train_auc = roc_auc_score(y_train, train_pred)
            val_auc = roc_auc_score(y_val, val_pred)
            print(train_auc, val_auc)
            print()
            
            results.append({
                'C': C,
                'solver': solver,
                'penalty': penalty,
                'train_auc': train_auc,
                'val_auc': val_auc
            })

# Convert results to DataFrame for easy analysis
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('val_auc', ascending=False)

print("Top 5 models by validation AUC:")
print(results_df.head())

# Get best parameters
best_params = results_df.iloc[0]
print("\nBest parameters:")
print(f"C={best_params['C']}, solver={best_params['solver']}, penalty={best_params['penalty']}")
print(f"Validation AUC: {best_params['val_auc']:.3f}")

# Train final model with best parameters
best_model = LogisticRegression(
    C=best_params['C'],
    solver=best_params['solver'],
    penalty=best_params['penalty'],
    random_state=1,
    max_iter=1000
)
best_model.fit(X_train, y_train)

0.0001 liblinear l1
0.38892291653091815 0.40761714412613814

0.0001 liblinear l2
0.6426174496644295 0.5664556962025318

0.0001 saga l1
0.38941161138984814 0.4088385520763935

0.0001 saga l2


/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.5469863817032645 0.4409282700421941

0.001 liblinear l1
0.4935133902391347 0.42926937597157444

0.001 liblinear l2
0.6563367433374601 0.5924938929602488

0.001 saga l1


/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.48886101518212033 0.4264934488119032

0.001 saga l2


...

0.5479279337981364 0.44120586275816126

10 liblinear l1
0.7219489150974132 0.5742282922496115

10 liblinear l2
0.7197302404378705 0.5600155451920943

10 saga l1


/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.5479149019352316 0.44115034421496785

10 saga l2
0.5479279337981364 0.44120586275816126

Top 5 models by validation AUC:
         C     solver penalty  train_auc   val_auc
12   0.100  liblinear      l1   0.674355  0.603015
5    0.001  liblinear      l2   0.656337  0.592494
9    0.010  liblinear      l2   0.685496  0.586387
8    0.010  liblinear      l1   0.645429  0.582667
20  10.000  liblinear      l1   0.721949  0.574228

Best parameters:
C=0.1, solver=liblinear, penalty=l1
Validation AUC: 0.603


/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(C=np.float64(0.1), max_iter=1000, penalty='l1',
                   random_state=1, solver='liblinear')

In [17]:
# Look at feature importance for the tuned model
feature_importance = pd.DataFrame({
    'feature': dv.feature_names_,
    'importance': best_model.coef_[0],
    'abs_importance': np.abs(best_model.coef_[0])
})
feature_importance = feature_importance.sort_values('abs_importance', ascending=False)

print('\nTop 10 most important features with tuned model:')
print(feature_importance.head(10))


Top 10 most important features with tuned model:
                               feature  importance  abs_importance
38         expected_student_count=<100    0.129421        0.129421
44                lead_source=Referral   -0.127615        0.127615
41    lead_source=Education Conference   -0.121131        0.121131
17            category_course_creation    0.089542        0.089542
60  primary_use_case=Employee Training   -0.084075        0.084075
50            organization_size=51-200    0.072476        0.072476
20                    category_support    0.061360        0.061360
2                 action_create_course    0.059987        0.059987
4                  action_invite_users    0.058763        0.058763
12           avg_hours_between_actions   -0.056850        0.056850


In [18]:
feature_names = dv.feature_names_

In [19]:
# Get all features related to lead_source
lead_source_features = [f for f in feature_names if 'lead_source=' in f]

# Get their coefficients
lead_source_coef = pd.DataFrame({
    'feature': lead_source_features,
    'coefficient': best_model.coef_[0][np.where([f in lead_source_features for f in feature_names])[0]]
})

# Sort by coefficient value to see most positive to most negative impact
lead_source_coef = lead_source_coef.sort_values('coefficient', ascending=False)

# Strip the 'lead_source=' prefix for cleaner display
lead_source_coef['source'] = lead_source_coef['feature'].str.replace('lead_source=', '')

print("Lead Source Impact on Conversion (sorted by coefficient):")
print(lead_source_coef[['source', 'coefficient']])

# You might also want to see the distribution of lead sources in your data
print("\nLead Source Distribution in Training Data:")
print(df_train['lead_source'].value_counts())

Lead Source Impact on Conversion (sorted by coefficient):
                 source  coefficient
0        Direct Traffic     0.000000
2         Google Search     0.000000
5          Social Media     0.000000
3   Product Review Site     0.000000
1  Education Conference    -0.121131
4              Referral    -0.127615

Lead Source Distribution in Training Data:
lead_source
Education Conference    326
Product Review Site     278
Referral                212
Social Media            182
Google Search           181
Direct Traffic          149
Name: count, dtype: int64
